In [1]:

from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt

In [3]:

df = pd.read_csv('weatherAUS.csv')

Hay tres clases atributos que observamos que tienen outliers, tenemos que decidir si son excepcionalidades, o casos de interés

In [19]:
pd.set_option('display.max_columns', None)

Cloud3pm,Cloud9am

Para medir la nubosidad, se mide los octavos de cielo cubirto, lo que implica que el rango de valores se tiene que encontrar entre 0 y 8. Sin embargo, nos hemos encontrado algunos registros donde la nubosidad ha sido 9. A veces, este valor valor se suele dar cuando no se puede observar el cielo (por ejemplo, por haber niebla). Observemos cómo son los registros de estas mediciones

In [20]:
df[(df["Cloud9am"]==9 )| (df["Cloud3pm"]==9)]

Date  Location  MinTemp  MaxTemp  Rainfall  Evaporation  \
30776   2009-09-23    Sydney     17.0     21.8       4.4         11.2   
47226   2012-05-27  Canberra      0.6     14.5       0.0          NaN   
106933  2012-11-02   Woomera     10.6     24.6       0.2         11.6   

        Sunshine WindGustDir  WindGustSpeed WindDir9am WindDir3pm  \
30776        8.0         NaN            NaN          W          W   
47226        7.8          SW           30.0        NNW        SSW   
106933      11.4         SSE           39.0         SE        ESE   

        WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  Pressure9am  \
30776           30.0          37.0         40.0         36.0        996.8   
47226            4.0           9.0         99.0         41.0       1022.4   
106933          26.0          17.0         45.0         13.0       1019.0   

        Pressure3pm  Cloud9am  Cloud3pm  Temp9am  Temp3pm RainToday  \
30776         997.8       9.0       3.0     17.3     20.9       Yes   
47226        1020.7       9.0       NaN      4.4     14.2        No   
106933       1015.2       6.0       9.0     14.8     23.7        No   

       RainTomorrow  
30776            No  
47226            No  
106933           No

Podemos observar que no hay nada fuera de lo común de estas mediciones, salvo su valor de nubosidad, así se que se considera´ra descartar estos registros

In [34]:
def IQR(atributo):
    Q1 = df[atributo].quantile(0.25)
    Q3 = df[atributo].quantile(0.75)

    # Step 2: Calculate IQR
    IQR = Q3 - Q1

    upper_bound = Q3 + 1.5 * IQR
    return upper_bound

Rainfall

Observando la gráfica, podemos suponer como outliers las medidciones con donde se ha observado una precipitación de más de 250mm :

<!-- Para analizar los outliers, observamos priemro las ditribuciones que hemos obtenido, y hay tres atributos con outliers: Cloud, Rainfall, y evaloaporation -->

In [40]:
total_registros = 145460

3/145460

In [72]:
3/145460 *100

0.002062422659150282

In [73]:
df.nlargest(int(total_registros/1000), 'Rainfall')[(df["RainTomorrow"]=="No")].count()

C:\Users\Acer\AppData\Local\Temp\ipykernel_13936\1430936761.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.nlargest(int(total_registros/1000), 'Rainfall')[(df["RainTomorrow"]=="No")].count()


Date             38
Location         38
MinTemp          37
MaxTemp          38
Rainfall         38
Evaporation       6
Sunshine         18
WindGustDir      37
WindGustSpeed    37
WindDir9am       38
WindDir3pm       37
WindSpeed9am     38
WindSpeed3pm     38
Humidity9am      38
Humidity3pm      38
Pressure9am      35
Pressure3pm      35
Cloud9am         27
Cloud3pm         26
Temp9am          38
Temp3pm          38
RainToday        38
RainTomorrow     38
dtype: int64

Estos datos corresponden a indundaciones o tormentas extremas. Por ejemplo, si buscamos "2011-02-16 Darwin flood" es internet, encontramos:

28% de no llover mañana, lo que implica que es un caso de relevancia, puesto que sugieren una alta probilidad de que llueva el día siguiente. Por tanto, conservaremos estos outliers

Evaporation

Tal y como se ha medido la evaporación, implica que, por condiciones físicas, aqullos valores superiores a 400 mm son muy singulares, incluso podría decirse imposibles.

In [84]:
df.nlargest(int(total_registros/1000), 'Evaporation')[(df["RainTomorrow"]=="Yes")].count()

C:\Users\Acer\AppData\Local\Temp\ipykernel_13936\2351061867.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.nlargest(int(total_registros/1000), 'Evaporation')[(df["RainTomorrow"]=="Yes")].count()


Date             14
Location         14
MinTemp          14
MaxTemp          14
Rainfall         14
Evaporation      14
Sunshine          7
WindGustDir      14
WindGustSpeed    14
WindDir9am       13
WindDir3pm       14
WindSpeed9am     13
WindSpeed3pm     14
Humidity9am      14
Humidity3pm      14
Pressure9am      14
Pressure3pm      14
Cloud9am         13
Cloud3pm         12
Temp9am          14
Temp3pm          14
RainToday        14
RainTomorrow     14
dtype: int64

En evaporación, el 0.1% de los registros con mayor evaporación medida son mayores a 35mm al día, que es un ínidice extremadamente alto, incluso para ambientes desérticos como es Autralia, así que se decide eliminarlos

In [82]:
df.nlargest(int(total_registros/1000), ['WindGustSpeed','WindSpeed9am','WindSpeed3pm'])[(df["RainTomorrow"]=="Yes")].count()

C:\Users\Acer\AppData\Local\Temp\ipykernel_13936\2406792113.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.nlargest(int(total_registros/1000), ['WindGustSpeed','WindSpeed9am','WindSpeed3pm'])[(df["RainTomorrow"]=="Yes")].count()


Date             90
Location         90
MinTemp          89
MaxTemp          90
Rainfall         88
Evaporation      55
Sunshine         52
WindGustDir      90
WindGustSpeed    90
WindDir9am       88
WindDir3pm       89
WindSpeed9am     88
WindSpeed3pm     89
Humidity9am      88
Humidity3pm      88
Pressure9am      79
Pressure3pm      78
Cloud9am         58
Cloud3pm         61
Temp9am          88
Temp3pm          88
RainToday        88
RainTomorrow     90
dtype: int64